<a href="https://colab.research.google.com/github/pau-torrente/capstone-project/blob/pau/tidy_file_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

# google drive mounting for running on colab

Mounted at /content/drive


In [2]:
# pip install required for the ml part of dask
# ! pip install dask_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 9.2 MB/s eta 0:00:00


In [5]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from dask_ml.preprocessing import StandardScaler, MinMaxScaler
from dask_ml.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from IPython.display import Markdown

####################################################################################################
# To run in colab

station_dataframe = dd.read_csv('/content/drive/MyDrive/CapstoneProject/data_bicing_joined_HX.csv', assume_missing=True, delimiter=';')

weather_dataframe = dd.read_csv('/content/drive/MyDrive/CapstoneProject/weather.csv', assume_missing=True, delimiter=',')

####################################################################################################

####################################################################################################

# To run locally

# station_dataframe = dd.read_csv('data/data_bicing_joined_HX.csv', assume_missing=True, delimiter=';')

# weather_dataframe = dd.read_csv('weather_data/weather.csv', assume_missing=True, delimiter=',')

####################################################################################################



#Initial cleaning to go from the format that the Tableau merger outputs to the one desired for the model
# The objective of this first step is to leave the training data in the same format as the submission one

# Drop all rows with out of service stations and year 2020, to avoid covid effects on our training data

station_dataframe = station_dataframe.loc[station_dataframe['status'] == 'IN_SERVICE']
station_dataframe = station_dataframe.loc[station_dataframe['year'] != 2020]

bare_df = station_dataframe[['station_id', 'lat', 'lon', 'year', 'month', 'day', 'hour', '% Docks Availlable',  '% Docks Available H-4','% Docks Available H-3', '% Docks Available H-2', '% Docks Available H-1']]
bare_df = bare_df.rename(columns={'% Docks Availlable': 'percentage'})
for i in range(1, 5):
    bare_df = bare_df.rename(columns={f'% Docks Available H-{i}': f'ctx-{i}'})

# Print the head of the updated DataFrame
bare_df = bare_df[['station_id', 'lat', 'lon', 'year', 'month', 'day', 'hour', 'ctx-4', 'ctx-3', 'ctx-2',	'ctx-1', 'percentage']]

bare_df.head()

,station_id,lat,lon,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,percentage
0,290.0,41.437338,2.174096,2019.0,7.0,22.0,8.0,0.352941,0.352941,0.352941,0.504902,0.751131
1,271.0,41.450608,2.192363,2022.0,6.0,10.0,21.0,0.753968,0.659341,0.645022,0.686508,0.769231
2,149.0,41.395868,2.192952,2022.0,6.0,8.0,20.0,0.491582,0.675214,0.864198,0.801347,0.735043
4,358.0,41.387244,2.179304,2021.0,5.0,28.0,8.0,0.480000,0.480000,0.513333,0.766667,0.943333
5,49.0,41.391076,2.196577,2022.0,5.0,27.0,8.0,0.761905,0.789683,0.809524,0.845238,0.913420


In [ ]:
#len(bare_df) = 14877478   To check if the number of columns gets modified

In [6]:
# dataframe to be predicted

data_2_predict = dd.read_csv('/content/drive/MyDrive/CapstoneProject/metadata_sample_submission.csv', delimiter=',')

# data_2_predict = dd.read_csv('data/metadata_sample_submission.csv', delimiter=',')
data_2_predict.head()

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160


In [9]:
# Import febrary 2023 station info to get the lat and lon of the stations --> March 2023 info could be used as well

####################################################################################################

# To run locally

# february2023_info = pd.read_csv('data/2023_02_Febrer_BicingNou_INFORMACIO.csv', delimiter=',')

####################################################################################################


####################################################################################################

# To run in colab

february2023_info = pd.read_csv('/content/drive/MyDrive/CapstoneProject/2023_02_Febrer_BicingNou_INFORMACIO.csv', delimiter=',')

####################################################################################################


final_loc = february2023_info.drop_duplicates(subset=['station_id'], keep='last')[['station_id', 'lat', 'lon']]

# february2023_info = pd.read_csv('/content/drive/MyDrive/CapstoneProject/march2023_info.cÇsv', delimiter=';')  # For running in colab

In [10]:
final_loc.head()

,station_id,lat,lon
1048556,500,41.411957,2.144752
3821777,520,41.386220,9.150292
3853425,494,41.389697,2.165220
4044289,21,41.410844,2.174057
4049296,515,41.435207,2.194800


In [11]:
def weather_prep(weather_df:dd) -> dd:

    # weather dataframe preparator to leave it in the desired format to merge with the station dataframe
    # We convert the half hourly data to hourly data by averaging temperature and summing precipitation
    # We also add a column with the datetime in order to merge the dataframes
    
    weather = weather_df.copy()

    weather = weather.groupby(weather.index//2).mean()
    
    weather['mm_precip'] = weather['mm_precip']*2
    weather['timestamp'] = (weather['timestamp']-900).astype(int)
    weather['datetime'] = weather['timestamp'].map(lambda x: pd.to_datetime(x, unit='s'))

    
    weather['timestamp'] = weather['timestamp'].astype(int)

    weather['datetime'] = weather['timestamp'].map(lambda x: pd.to_datetime(x, unit='s'))
    
    return weather

####################################################################################################

def weather_merge(weather_df:dd, station_data:dd) -> dd:
    weather = weather_df.copy()
    stations = station_data.copy()
    if 'year' not in stations.columns:
        stations['year'] = 2023 # create a year column to make the sumbission data equivalent to the training one

    stations[['year', 'month', 'day', 'hour']] = stations[['year', 'month', 'day', 'hour']].astype(int)

    stations['datetime'] = dd.to_datetime(stations['year'].astype(str) + '-' +
                                                stations['month'].astype(str) + '-' +
                                                stations['day'].astype(str) + ' ' +
                                                stations['hour'].astype(str) + ':00:00') # create a datetime column to merge with the weather dataframe
    
    # Merge for the current datetime
    stations = dd.merge(left=stations, right=weather[['datetime', 'temperature','mm_precip']], left_on='datetime', right_on='datetime', how='left')

    # Iterate to merge for the preceding datetimes

    for i in range(1,5):
        df_weather_shifted = weather.copy()
        df_weather_shifted['datetime'] = df_weather_shifted['datetime'] + pd.Timedelta(hours=-i)
        stations = stations.merge(df_weather_shifted[['datetime', 'temperature','mm_precip']], on='datetime', how='inner', suffixes=('', f'-{abs(i)}'))

    return stations

####################################################################################################
####################################################################################################

# According to chatGPT, the following function should solve the problem of having missmatching number of rows --> Give it a try


# def weather_merge(weather_df: dd, station_data: dd) -> dd:
#     weather = weather_df.copy()
#     stations = station_data.copy()
    
#     if 'year' not in stations.columns:
#         stations['year'] = 2023
    
#     stations[['year', 'month', 'day', 'hour']] = stations[['year', 'month', 'day', 'hour']].astype(int)
    
#     stations['datetime'] = dd.to_datetime(stations['year'].astype(str) + '-' +
#                                           stations['month'].astype(str) + '-' +
#                                           stations['day'].astype(str) + ' ' +
#                                           stations['hour'].astype(str) + ':00:00')
    
#     stations = dd.merge(left=stations, right=weather[['datetime', 'temperature','mm_precip']], left_on='datetime', right_on='datetime', how='left')
    
#     df_weather_shifted = weather.copy()
    
#     for i in range(1, 5):
#         df_weather_shifted['datetime_shifted'] = df_weather_shifted['datetime'] + pd.Timedelta(hours=-i)
#         suffix = f'-{abs(i)}'
#         stations = stations.merge(df_weather_shifted[['datetime_shifted', 'temperature', 'mm_precip']], left_on='datetime', right_on='datetime_shifted', how='left', suffixes=('', suffix))
#         stations = stations.drop('datetime_shifted', axis=1)
    
#     return stations

####################################################################################################
####################################################################################################

class weather_merge_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, weather_df:dd, merging_function:callable=weather_merge):
        self.weather_df = weather_df
        self.func = merging_function

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.func(self.weather_df, X)
    
####################################################################################################


def extra_time_info(df:dd) -> dd:

    def is_weekend(day_of_week):
        return 1 if day_of_week >= 5 else 0

    # Create a column to distinguish id ii's a weekend or not
    df['is_weekend'] = df['datetime'].dt.dayofweek.map(is_weekend, meta=('is_weekend', 'int64'))

    # Since we will only be working with 5 hours, we add 5 binary columns to indicate in which part of the day we are
    # to sort of solve the problem of the missing hours for training

    df['timeframe1'] = df['datetime'].dt.hour.map(lambda x: 1 if x <= 4 else 0, meta=('timeframe1', 'int64'))
    df['timeframe2'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 5 and x <=9 else 0, meta=('timeframe1', 'int64'))
    df['timeframe3'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 10 and x <=14 else 0, meta=('timeframe1', 'int64'))
    df['timeframe4'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 15 and x <=19 else 0, meta=('timeframe1', 'int64'))
    df['timeframe5'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 20 else 0, meta=('timeframe1', 'int64'))

    df = df.drop(['datetime'], axis = 1)
    
    return df

####################################################################################################


def station_loc(id_lat_lon:dd, df:dd) -> dd: #this should be applied for the 2023 march dataset to predict from station_id-location paris from february 2023

    # station location adder from the february 2023 dataset to substitute station_id for locations

    assert all(item in list(id_lat_lon.columns) for item in ['station_id', 'lat', 'lon']), 'id_lat_lon must contain station_id, lat and lon columns'
    id_locator = id_lat_lon.copy()
    data = df.copy()
    id_locator = id_locator.drop_duplicates(subset=['station_id'])

    data = data.merge(id_locator[['station_id', 'lat', 'lon']], on='station_id', how='left')

    return data

class station_loc_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, id_lat_lon:dd, merging_function:callable=station_loc):
        self.id_lat_lon = id_lat_lon
        self.func = merging_function

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return self.func(self.id_lat_lon, X)
    
####################################################################################################
    

    
def hour_selector(df:dd, hour_list:list) -> dd:

    # Hour selector function to select only the hours we want to train on
    # in our case we will only train on 5 hours of the day, but a more complicated function
    # that would cover all hours could also be constructed

    data = df.copy()
    data = data.loc[data['hour'].isin(hour_list)]
    return data

class hour_selector_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, hour_list:list):
        self.hour_list = hour_list

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return hour_selector(X, self.hour_list)

####################################################################################################


def time_norm(df:dd, columns:list) -> dd:
    # Time normalizer function to normalize the time columns to a 0-1 scale with periodicity
    # This is done by applying a sin and cos transformation to the columns

    data = df.copy()
    for col in columns:
        data['cos_'+col] = np.cos(2*np.pi*data[col]/data[col].max())
        data['sin_'+col] = np.sin(2*np.pi*data[col]/data[col].max())
        data = data.drop([col], axis=1)

    data['year_normed'] = (data['year']-2019)/4
    data = data.drop(['year'], axis = 1)
    
    return data

class time_norm_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns: list, norm_function: callable=time_norm):
        self.columns = columns
        self.func = norm_function

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return self.func(X, self.columns)
    
####################################################################################################
    
class station_id_dropper(BaseEstimator, TransformerMixin):

    # Station id dropper function to drop the station_id column from the dataset at the end of the pipeline
    # prediction will be done on the location of the stations, not on the id
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(['station_id'], axis=1)

In [ ]:
weather_prepped = weather_prep(weather_dataframe)

weather_prepped.head()

,timestamp,mm_precip,temperature,datetime
0,1546297200,0.0,9.00,2018-12-31 23:00:00
1,1546300800,0.0,9.10,2019-01-01 00:00:00
2,1546304400,0.0,8.45,2019-01-01 01:00:00
3,1546308000,0.0,7.55,2019-01-01 02:00:00
4,1546311600,0.0,7.50,2019-01-01 03:00:00


In [ ]:
# Creation of the 3 pipelines:
# - train_preparator: to prepare the training data, not used for the submission data as we already have location data
# - submisison_preparator: to prepare the submission data, not used for the training data as we don't have location data --> Not a problem since we are not fitting to any data, just transformitng with data we already have
# - scaling_pipeline: to fill and scale certain column in the data, used for both training and submission data --> In this case, we are fitting to the training data, and transforming both training and submission data 

traintest_preparator = Pipeline([
    ('weather_merge', weather_merge_transformer(weather_df=weather_prepped)),  # weather_merge transformer already includes the creation of the datetime column
    ('extra_time_info', FunctionTransformer(func=extra_time_info)),
    ('hour_selector', hour_selector_transformer(hour_list=[4, 9, 14, 19, 23])),
    ('time_normalization', time_norm_transformer(columns=['month', 'day', 'hour'])),
    ('station_id_dropper', station_id_dropper())
])

submission_preparator = Pipeline([
    ('weather_merge', weather_merge_transformer(weather_df=weather_prepped)),  # weather_merge transformer already includes the creation of the datetime column'
    ('extra_time_info', FunctionTransformer(func=extra_time_info)),
    ('hour_selector', hour_selector_transformer(hour_list=[i for i in range(24)])),
    ('time_normalization', time_norm_transformer(columns=['month', 'day', 'hour'])),
    ('station_loc', station_loc_transformer(id_lat_lon=final_loc)),
    ('station_id_dropper', station_id_dropper())    
])



total_columns = ['lat', 'lon', 'ctx-4', 'ctx-3', 'ctx-2', 'ctx-1',
       'temperature', 'mm_precip', 'temperature-1', 'mm_precip-1',
       'temperature-2', 'mm_precip-2', 'temperature-3', 'mm_precip-3',
       'temperature-4', 'mm_precip-4', 'is_weekend', 'timeframe1',
       'timeframe2', 'timeframe3', 'timeframe4', 'timeframe5', 'cos_month',
       'sin_month', 'cos_day', 'sin_day', 'cos_hour', 'sin_hour',
       'year_normed']


to_fill = ['ctx-4', 'ctx-3', 'ctx-2', 'ctx-1', 'is_weekend', 'timeframe1',
                   'timeframe2', 'timeframe3', 'timeframe4', 'timeframe5', 'cos_month',
                   'sin_month', 'cos_day', 'sin_day', 'cos_hour', 'sin_hour',
                   'year_normed']


to_fill_and_scale = ['lat', 'lon', 'temperature', 'mm_precip', 'temperature-1', 'mm_precip-1',
       'temperature-2', 'mm_precip-2', 'temperature-3', 'mm_precip-3',
       'temperature-4', 'mm_precip-4']


filler = Pipeline([
    ('nan_filler', SimpleImputer(missing_values = np.nan, strategy='median'))
])

filler_scaler= Pipeline([
    ('nan_filler', SimpleImputer(missing_values = np.nan, strategy='median')),
    ('min_max', MinMaxScaler(feature_range = (-1,1)))
])

scaling_pipeline = ColumnTransformer([
    ('fill', filler, to_fill),
    ('fill_scale', filler_scaler, to_fill_and_scale)
], remainder='drop')


In [ ]:
# Complete the data for training/validation
data_prepared = traintest_preparator.fit_transform(bare_df)
data_prepared.compute().to_csv('prepped_data/full_data.csv', index=False)
# print(data_prepared.shape[0].compute())   = 3101267  Value from a previous run



In [ ]:
# Separate the training/validation data into X and y, apply the pipeline and save them to csv files
y = data_prepared['percentage'].values.compute()
ydf = pd.DataFrame({'percentage': y})
ydf.to_csv('prepped_data/y_t_traintest.csv', index=False)

X = pd.DataFrame(scaling_pipeline.fit_transform(data_prepared), columns= to_fill + to_fill_and_scale)
X.to_csv('prepped_data/X_to_traintest.csv', index=False)

In [ ]:
# Complete the data for submission and sabe it to a csv file
data_2_predict_prepared = submission_preparator.fit_transform(data_2_predict)
data_2_predict_prepared.compute().to_csv('prepped_data/full_data_2_predict.csv', index=False)



In [ ]:
# Apply the pipeline to the submission data and save it to a csv file
X_submission = pd.DataFrame(scaling_pipeline.transform(data_2_predict_prepared), columns= to_fill + to_fill_and_scale)
X_submission.to_csv('prepped_data/X_submission.csv', index=False)

In [ ]:
len(data_prepared)

14873854

In [ ]:
print(data_prepared.head().to_markdown(tablefmt = 'fancy_grid'))


╒════╤═════════╤═══════════════╤═════════════╤═════════════╤═══════════╤═══════════╤════════════╤════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤═════════╤═════════╤═════════════════╤═══════════════╤══════════╤═════════════════╤═══════════════╤═══════════╤═════════════════╤═══════════════╤══════════╤═════════════════╤═══════════════╤══════════╤═══════════════╤═════════════╤══════════════╕
│    │   index │   year_normed │   cos_month │   sin_month │   cos_day │   sin_day │   cos_hour │   sin_hour │   is_weekend │   timeframe1 │   timeframe2 │   timeframe3 │   timeframe4 │   timeframe5 │     lat │     lon │   temperature-4 │   mm_precip-4 │    ctx-4 │   temperature-3 │   mm_precip-3 │     ctx-3 │   temperature-2 │   mm_precip-2 │    ctx-2 │   temperature-1 │   mm_precip-1 │    ctx-1 │   temperature │   mm_precip │   percentage │
╞════╪═════════╪═══════════════╪═════════════╪═════════════╪═══════════╪═══════════╪════════════╪═════════

# XGBoost model
## Trained a few days ago before all the preprocessing that now is being done with the pipeline

Was used to test wheter the use of this model made sense, with a very small sample of data -> Now could be scaled

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

reduced_data_prepared = data_prepared.head(1000)

reduced_data_prepared = reduced_data_prepared[reduced_data_prepared['hour'].isin([4,9,14,19,23])]

X = reduced_data_prepared.drop(['percentage'], axis=1)
y = reduced_data_prepared['percentage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)



In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

space = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0],
    'reg_alpha': [0, 0.1, 1.0],
    'n_estimators': [100, 200, 300]
}

xgb_model = XGBRegressor(objective='reg:squarederror', random_state = 69)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=space, scoring='neg_mean_squared_error', cv=3, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 729 candidates, totalling 2187 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=6

In [ ]:
import json

# Trained the model and saved it to a file

parameter = open('models/xgboost1000.json.json', 'w+')
parameter.write(json.dumps(grid_search.best_params_))
parameter.close()


In [ ]:
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=6, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, gamma=0.1, reg_alpha=0.1, reg_lambda=0.1, n_jobs=-1, random_state=123)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'RMSE: {rmse}')

# BILSTM trials


In [20]:
# recover the prepared data from the csv files
X_data = pd.read_csv('/content/drive/MyDrive/CapstoneProject/prepped_data/X_to_traintest.csv', delimiter = ',')
y_data = pd.read_csv('/content/drive/MyDrive/CapstoneProject/prepped_data/y_t_traintest.csv', delimiter = ',')

In [37]:
X_data.head()

,ctx-4,ctx-3,ctx-2,ctx-1,is_weekend,timeframe1,timeframe2,timeframe3,timeframe4,timeframe5,...,temperature,mm_precip,temperature-1,mm_precip-1,temperature-2,mm_precip-2,temperature-3,mm_precip-3,temperature-4,mm_precip-4
0,0.749288,0.814815,0.831909,0.882155,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.171429,-1.0,-0.189802,-1.0,-0.196557,-1.0,-0.17765,-1.0,-0.102962,-1.0
1,0.008547,0.356902,0.236467,0.097643,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.171429,-1.0,-0.189802,-1.0,-0.196557,-1.0,-0.17765,-1.0,-0.102962,-1.0
2,0.824176,0.883117,0.663004,0.415584,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.171429,-1.0,-0.189802,-1.0,-0.196557,-1.0,-0.17765,-1.0,-0.102962,-1.0
3,0.910256,0.848485,0.855769,0.530303,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.171429,-1.0,-0.189802,-1.0,-0.196557,-1.0,-0.17765,-1.0,-0.102962,-1.0
4,0.752137,0.784512,0.669516,0.663300,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.171429,-1.0,-0.189802,-1.0,-0.196557,-1.0,-0.17765,-1.0,-0.102962,-1.0


In [21]:
X_data.columns

Index(['ctx-4', 'ctx-3', 'ctx-2', 'ctx-1', 'is_weekend', 'timeframe1',
       'timeframe2', 'timeframe3', 'timeframe4', 'timeframe5', 'cos_month',
       'sin_month', 'cos_day', 'sin_day', 'cos_hour', 'sin_hour',
       'year_normed', 'lat', 'lon', 'temperature', 'mm_precip',
       'temperature-1', 'mm_precip-1', 'temperature-2', 'mm_precip-2',
       'temperature-3', 'mm_precip-3', 'temperature-4', 'mm_precip-4'],
      dtype='object')

In [22]:
y_data.head()

,percentage
0,0.732194
1,0.088319
2,0.267399
3,0.179487
4,0.672365


In [36]:
X_submission = pd.read_csv('/content/drive/MyDrive/CapstoneProject/prepped_data/X_submission.csv', delimiter = ',')
X_submission.head()

,ctx-4,ctx-3,ctx-2,ctx-1,is_weekend,timeframe1,timeframe2,timeframe3,timeframe4,timeframe5,...,temperature,mm_precip,temperature-1,mm_precip-1,temperature-2,mm_precip-2,temperature-3,mm_precip-3,temperature-4,mm_precip-4
0,0.753086,0.780864,0.799383,0.824074,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.205714,-1.0,-0.155807,-1.0,-0.07604,-1.0,-0.131805,-1.0,-0.046544,-1.0
1,0.629630,0.669753,0.750000,0.910494,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.205714,-1.0,-0.155807,-1.0,-0.07604,-1.0,-0.131805,-1.0,-0.046544,-1.0
2,1.000000,1.000000,0.962963,0.564815,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.205714,-1.0,-0.155807,-1.0,-0.07604,-1.0,-0.131805,-1.0,-0.046544,-1.0
3,0.814815,0.854938,0.938272,0.728395,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.205714,-1.0,-0.155807,-1.0,-0.07604,-1.0,-0.131805,-1.0,-0.046544,-1.0
4,0.370370,0.367284,0.515432,0.589506,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.205714,-1.0,-0.155807,-1.0,-0.07604,-1.0,-0.131805,-1.0,-0.046544,-1.0


In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
# Assuming your dataframe is named 'df'
# Extract the static data, the target and the time series data in seperate arrays

static_data_columns = ['is_weekend', 'timeframe1',
       'timeframe2', 'timeframe3', 'timeframe4', 'timeframe5', 'cos_month',
       'sin_month', 'cos_day', 'sin_day', 'cos_hour', 'sin_hour',
       'year_normed', 'lat', 'lon', 'temperature', 'mm_precip']
static_data = X_data[static_data_columns].values

timestep_1 = X_data[['temperature-1', 'mm_precip-1', 'ctx-1']].values
timestep_2 = X_data[['temperature-2', 'mm_precip-2', 'ctx-2']].values
timestep_3 = X_data[['temperature-3', 'mm_precip-3', 'ctx-3']].values
timestep_4 = X_data[['temperature-4', 'mm_precip-4', 'ctx-4']].values

time_series_data = np.stack((timestep_4, timestep_3, timestep_2, timestep_1), axis=1)

target = y_data[['percentage']].values
# timestep_1 = np.column_stack((data_prepared[['temperature-1', 'mm_precip-1', 'ctx-1']].values, static_data))
# timestep_2 = np.column_stack((data_prepared[['temperature-2', 'mm_precip-2', 'ctx-2']].values, static_data))
# timestep_3 = np.column_stack((data_prepared[['temperature-3', 'mm_precip-3', 'ctx-3']].values, static_data))
# timestep_4 = np.column_stack((data_prepared[['temperature-4', 'mm_precip-4', 'ctx-4']].values, static_data))




In [29]:
static_data[0]


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00, -8.66025404e-01,  5.00000000e-01,
        1.51427778e-01,  9.88468324e-01,  1.00000000e+00, -2.44929360e-16,
        5.00000000e-01, -5.49637358e-01, -1.97289170e-01, -1.71428571e-01,
       -1.00000000e+00])

In [31]:
import tensorflow as tf
from keras.layers import LSTM, Dense, Input, concatenate, Dropout, Bidirectional

# Construct eh neural network by separating it into two parts, one for the time series data and one for the static data
# The time series data is fed into an LSTM layer, the static data is fed into a dense layer

ts_input = tf.keras.Input(shape=(4, 3), name='ts_input')
static_input = tf.keras.Input(shape = (17,), name='static_input')

####################################################################################################

LSTM1 = LSTM(64, activation='tanh', return_sequences=True)(ts_input)
dropout_lstm = Dropout(0.2)(LSTM1)
LSTM2 = LSTM(64, activation = 'tanh', return_sequences = False)(dropout_lstm)
lstm_out = Dropout(0.2)(LSTM2)

####################################################################################################

static1 = Dense(32, activation='tanh')(static_input)
dropout_static = Dropout(0.2)(static1)
static2 = Dense(16, activation = 'tanh')(dropout_static)
static_out = Dropout(0.2)(static2)

####################################################################################################

merged_out = concatenate([lstm_out, static_out]) # join the outputs of the LSTM and dense layers and pass them into a final neuron
merged_out = Dense(1, activation='tanh')(merged_out)

model = tf.keras.Model(inputs=[ts_input, static_input], outputs=merged_out)

model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ts_input (InputLayer)          [(None, 4, 3)]       0           []                               
                                                                                                  
 static_input (InputLayer)      [(None, 17)]         0           []                               
                                                                                                  
 lstm_7 (LSTM)                  (None, 4, 64)        17408       ['ts_input[0][0]']               
                                                                                                  
 dense_11 (Dense)               (None, 32)           576         ['static_input[0][0]']           
                                                                                            

In [32]:
from keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

# Now fit the model to the data, obtaining pretty good results, save the model to a file and then predict on the X_submission data

model.fit(x = [time_series_data, static_data],
          y = target,
          batch_size = 128,
          epochs = 25,
          validation_split=.2,
          callbacks=[earlyStopping],
          shuffle=True)

Epoch 1/25
14797/14797 [==============================] - 155s 10ms/step - loss: 0.0143 - mse: 0.0143 - val_loss: 0.0127 - val_mse: 0.0127
Epoch 2/25
14797/14797 [==============================] - 138s 9ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.0124 - val_mse: 0.0124
Epoch 3/25
14797/14797 [==============================] - 150s 10ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.0124 - val_mse: 0.0124
Epoch 4/25
14797/14797 [==============================] - 138s 9ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.0122 - val_mse: 0.0122
Epoch 5/25
14797/14797 [==============================] - 123s 8ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.0122 - val_mse: 0.0122
Epoch 6/25
14797/14797 [==============================] - 116s 8ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.0121 - val_mse: 0.0121
Epoch 7/25
14797/14797 [==============================] - 116s 8ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.0121 - val_mse: 0.0121
Epoch 8/25
14797/14797 [========

In [33]:
model.save('/content/drive/MyDrive/CapstoneProject/keras_models/model_1')

In [34]:
model.save('/content/drive/MyDrive/CapstoneProject/keras_models/model_1_h5.h5')

In [38]:
static_sub = X_submission[static_data_columns].values

timestep_1_sub = X_submission[['temperature-1', 'mm_precip-1', 'ctx-1']].values
timestep_2_sub = X_submission[['temperature-2', 'mm_precip-2', 'ctx-2']].values
timestep_3_sub = X_submission[['temperature-3', 'mm_precip-3', 'ctx-3']].values
timestep_4_sub = X_submission[['temperature-4', 'mm_precip-4', 'ctx-4']].values

time_series_sub = np.stack((timestep_4_sub, timestep_3_sub, timestep_2_sub, timestep_1_sub), axis=1)

In [39]:
y_pred = model.predict(x = [time_series_sub, static_sub])

1710/1710 [==============================] - 6s 3ms/step


In [47]:
prediction = pd.DataFrame({'percentage_docks_available':y_pred.flatten()})

In [54]:

prediction.to_csv('/content/drive/MyDrive/CapstoneProject/keras_models/model_1_prediction.csv', index = True)

In [46]:
y_pred.shape

(54690, 1)

List of thing to do:
1. Encode month/hour as cyclical features or simply remove them
2. standarize percentages to max
3. standarize temperatures and rain with min max scaler
4. look into how to train with data from multiple hours and not just intervals of 4
5. F******* TRAIN SOME MODELS 
